In [ ]:
import csv

In [ ]:
# get the data in a format our training script can use
# https://www.uvic.ca/engineering/ece/isot/datasets/fake-news/index.php
with open('../../News _dataset/Fake.csv', 'r', encoding='utf-8') as fake:
    with open('../../News _dataset/True.csv', 'r', encoding='utf-8') as real:
        with open('./datasets/ISOT_FakeNews.csv', 'w+', newline='', encoding='utf-8') as out:
            fakeReader = csv.reader(fake, delimiter=',')
            realReader = csv.reader(real, delimiter=',')
            next(fakeReader)
            next(realReader)
            output = csv.writer(out, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            output.writerow(['label', 'headline', 'text'])
            for line in fakeReader:
                output.writerow(['1', line[0], line[1]])
            for line in realReader:
                output.writerow(['0', line[0], line[1]])

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('datasets/ISOT_FakeNews.csv')

In [ ]:
# shuffle data and generate our train valid test split
train, valid, test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

In [ ]:
print(len(df))
print(len(train), len(valid), len(test))

In [ ]:
train.to_csv('datasets/ISOT_train.csv', index=False)
valid.to_csv('datasets/ISOT_valid.csv', index=False)
test.to_csv('datasets/ISOT_test.csv', index=False)

In [10]:
import datasets

In [11]:
train_file = './datasets/ISOT_train.csv'
eval_file = './datasets/ISOT_valid.csv'
test_file = './datasets/ISOT_test.csv'
files = {}
if train_file is not None:
    files[datasets.Split.TRAIN] = [train_file]
if eval_file is not None:
    files[datasets.Split.VALIDATION] = [eval_file]
if test_file is not None:
    files[datasets.Split.TEST] = [test_file]

ds = datasets.load_dataset("csv", data_files=files, quotechar='"')
features_name = list(ds[list(files.keys())[0]].features.keys())
label_name = features_name.pop(0)
label_list = list(set(ds[list(files.keys())[0]][label_name]))
label2id = {label: i for i, label in enumerate(label_list)}
# input_names = tokenizer.model_input_names
transformed_ds = {}

Using custom data configuration default-9b9f86091f19c68a


Dataset csv downloaded and prepared to C:\Users\ethan\.cache\huggingface\datasets\csv\default-9b9f86091f19c68a\0.0.0\2a88c45fed596f9421a2e7f74ab1a3cd012ef75210a5dc1950e8d60ca8d9c66c. Subsequent calls will reuse this data.


In [31]:
# test_ds = ds['test'].select(range(10))
for x in test_ds:
    print(type(x[features_name[1]]))

test_ds.map(lambda x: print(type(x[features_name[1]])), batched=True)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'list'>


<class 'list'>


Dataset({
    features: ['label', 'headline', 'text'],
    num_rows: 10
})

In [12]:
from transformers import AutoTokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [34]:
def plswork(example):
    # print(type(example[features_name[1]]))
    try:
        ret = tokenizer.batch_encode_plus(
            (example[features_name[0]], example[features_name[1]]),
            truncation=True,
            max_length=512,
            padding="max_length",
        )
    except Exception as e:
        # print(example[features_name[0]])
        print(len(example[features_name[0]]))
        raise e

In [35]:
transformed_ds['test'] = ds['test'].select(range(10)).map(
    plswork,
    batched=True,
)

2


TypeError: batch_text_or_text_pairs has to be a list (got <class 'tuple'>)